In [ ]:
import networkx as nx
import numpy as np

import pystac_client
from oxeo.core.models.tile import load_tile_from_stac_as_dict, tile_from_id
from oxeo.core.models import data
URL = "https://earth-search.aws.element84.com/v0"
catalog = pystac_client.Client.open(URL)

In [ ]:
# Let's get an AOI to work with

search_params = {
                 "bbox":[-66.9235610961914, 
                         -39.76210275375138, 
                         -66.75533294677734, 
                         -39.64865819884837], #min_x, min_y, max_x, max_y
                 "collections":["sentinel-s2-l2a-cogs"],
                 "datetime":"2020-04-01/2020-05-01"
                 }
aoi = data.get_aoi_from_stac_catalog(catalog_url=URL, 
                                     search_params=search_params,
                                     chunk_aligned=True)
coords = aoi.isel(time=0).sel(band=["B04", "B03", "B02"]).coords


In [ ]:
from oxeo.core.models import graph 

In [ ]:
G = nx.Graph()
# Create graph with empty schema
G.add_nodes_from([
    ("wb_1", graph.NODE_SCHEMA),
    ("wb_2", graph.NODE_SCHEMA),
    ("asset_1", {"name": "ARegion"})
])
G.add_edge("wb_1", "asset_1", rel="isin")
G.add_edge("wb_2", "asset_1", rel="isin")

In [ ]:
# Append stackstac xa attrs to graph nodes
coords_dict = graph.stackstac_xa_coords_to_dict(coords)
graph.append_attrs(G, 'wb_1', attrs=coords_dict, appendable_attrs=["time", "res_level"])
graph.append_attrs(G, 'wb_2', attrs=coords_dict, appendable_attrs=["time", "res_level"])

# append res_level attrs to wb_1
graph.append_attrs(G, 'wb_1', attrs={"res_level":2}, appendable_attrs=["time", "res_level"])
graph.append_attrs(G, 'wb_1', attrs={"res_level":10}, appendable_attrs=["time", "res_level"])

# append res_level attrs to wb_2
graph.append_attrs(G, 'wb_2', attrs={"res_level":2}, appendable_attrs=["time", "res_level"])
graph.append_attrs(G, 'wb_2', attrs={"res_level":3}, appendable_attrs=["time", "res_level"])

In [ ]:
# Get the avg 
asset_avg_res_levels = []
for wb in list(G.neighbors("asset_1")):
    asset_avg_res_levels.append(np.mean(G.nodes[wb]["res_level"]))
    
asset_avg_res_levels